<a href="https://colab.research.google.com/github/mohamed97-ba/third_place_-Fraud-Detection-in-Electricity-and-Gas-Consumption-Challenge_solution/blob/master/Fraud_Detection_in_Electricity_and_Gas_Consumption_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fraud Detection in Electricity and Gas Consumption Challenge
The Tunisian Company of Electricity and Gas (STEG) is a public and a non-administrative company, it is responsible for delivering electricity and gas across Tunisia. The company suffered tremendous losses in the order of 200 million Tunisian Dinars due to fraudulent manipulations of meters by consumers.

Using the client’s billing history, the aim of the challenge is to detect and recognize clients involved in fraudulent activities.

The solution will enhance the company’s revenues and reduce the losses caused by such fraudulent activities. 
[about STEG:](https://www.steg.com.tn/en/institutionnel/mission.html)

![alt text](https://drive.google.com/uc?id=1nVmfP6Wu8Np19u7x2azbVO4C_3YpyiAU)

In [0]:
!unzip /content/train1.zip

unzip:  cannot find or open /content/train1.zip, /content/train1.zip.zip or /content/train1.zip.ZIP.


In [0]:
import numpy as np
import pandas as pd
import datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')
np.random.seed(4590)

# Nouvelle section

In [0]:
train_client=pd.read_csv('/content/client_train.csv')
test_client=pd.read_csv('/content/client_test.csv')
train_invoice=pd.read_csv('/content/invoice_train.csv')
test_invoice=pd.read_csv('/content/invoice_test.csv')
sub=pd.read_csv('/content/SampleSubmission.csv')

In [0]:
train_client.head()

,disrict,client_id,client_catg,region,creation_date,target
0,60,train_Client_0,11,101,31/12/1994,0.0
1,69,train_Client_1,11,107,29/05/2002,0.0
2,62,train_Client_10,11,301,13/03/1986,0.0
3,69,train_Client_100,11,105,11/07/1996,0.0
4,62,train_Client_1000,11,303,14/10/2014,0.0


In [0]:
train_invoice.head(10)

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type
0,train_Client_0,2014-03-24,11,1335667,0,203,8,1,82,0,0,0,14302,14384,4,ELEC
1,train_Client_0,2013-03-29,11,1335667,0,203,6,1,1200,184,0,0,12294,13678,4,ELEC
2,train_Client_0,2015-03-23,11,1335667,0,203,8,1,123,0,0,0,14624,14747,4,ELEC
3,train_Client_0,2015-07-13,11,1335667,0,207,8,1,102,0,0,0,14747,14849,4,ELEC
4,train_Client_0,2016-11-17,11,1335667,0,207,9,1,572,0,0,0,15066,15638,12,ELEC
5,train_Client_0,2017-07-17,11,1335667,0,207,9,1,314,0,0,0,15638,15952,8,ELEC
6,train_Client_0,2018-12-07,11,1335667,0,207,9,1,541,0,0,0,15952,16493,12,ELEC
7,train_Client_0,2019-03-19,11,1335667,0,207,9,1,585,0,0,0,16493,17078,8,ELEC
8,train_Client_0,2011-07-22,11,1335667,0,203,9,1,1200,186,0,0,7770,9156,4,ELEC
9,train_Client_0,2011-11-22,11,1335667,0,203,6,1,1082,0,0,0,9156,10238,4,ELEC


In [0]:
d={"ELEC":0,"GAZ":1}
train_invoice['counter_type']=train_invoice['counter_type'].map(d)

In [0]:
train_client['client_catg'] = train_client['client_catg'].astype('object')
train_client['disrict'] = train_client['disrict'].astype('object')

test_client['client_catg'] = test_client['client_catg'].astype('object')
test_client['disrict'] = test_client['disrict'].astype('object')

In [0]:
train_invoice['counter_type'].value_counts()

0    3079406
1    1397343
Name: counter_type, dtype: int64

In [0]:
for df in [train_invoice,test_invoice]:
    df['invoice_date'] = pd.to_datetime(df['invoice_date'])
    df['year'] = df['invoice_date'].dt.year
    df['month'] = df['invoice_date'].dt.month


In [0]:
aggs = {}
aggs['consommation_level_1'] = ['sum','max','min','mean','std']
aggs['consommation_level_2'] = ['sum','max','min','mean','std']
aggs['consommation_level_3'] = ['sum','max','min','mean','std']
aggs['consommation_level_4'] = ['sum','max','min','mean','std']

aggs['month'] = ['mean', 'max', 'min', 'std']
aggs['year'] = ['nunique','max','min','mean']

aggs['months_number'] = ['max','min','mean','sum']
aggs['reading_remarque'] = ['max','min','mean','std','sum']
aggs['counter_coefficient'] = ['max', 'min','mean']
aggs['counter_number'] = ['nunique']
aggs['counter_type'] = ['nunique','mean','sum']
aggs['counter_statue'] = ['nunique']
aggs['tarif_type'] = ['nunique','max','min']
aggs['counter_code'] = ['nunique','max','mean','min']


aggs['old_index'] = ['nunique','mean','std']
aggs['old_index'] = ['nunique','mean','std']





In [0]:
    agg_trans = train_invoice.groupby(['client_id']).agg(aggs)
    agg_trans.columns = ['_'.join(col).strip() for col in agg_trans.columns.values]
    agg_trans.reset_index(inplace=True)

    df = (train_invoice.groupby('client_id')
          .size()
          .reset_index(name='{}transactions_count'.format('1')))

    agg_trans = pd.merge(df, agg_trans, on='client_id', how='left')

In [0]:
agg_trans.head()

,client_id,1transactions_count,consommation_level_1_sum,consommation_level_1_max,consommation_level_1_min,consommation_level_1_mean,consommation_level_1_std,consommation_level_2_sum,consommation_level_2_max,consommation_level_2_min,consommation_level_2_mean,consommation_level_2_std,consommation_level_3_sum,consommation_level_3_max,consommation_level_3_min,consommation_level_3_mean,consommation_level_3_std,consommation_level_4_sum,consommation_level_4_max,consommation_level_4_min,consommation_level_4_mean,consommation_level_4_std,month_mean,month_max,month_min,month_std,year_max,year_min,year_mean,months_number_max,months_number_min,months_number_mean,months_number_sum,reading_remarque_max,reading_remarque_min,reading_remarque_mean,reading_remarque_std,reading_remarque_sum,counter_coefficient_max,counter_coefficient_min,counter_coefficient_mean,counter_number_nunique,counter_type_nunique,counter_type_mean,counter_type_sum,counter_statue_nunique,tarif_type_nunique,tarif_type_max,tarif_type_min,counter_code_nunique,counter_code_max,counter_code_mean,counter_code_min,old_index_nunique,old_index_mean,old_index_std
0,train_Client_0,35,12334,1200,38,352.400000,310.343472,370,186,0,10.571429,43.568935,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,6.685714,12,1,3.587326,2019,2005,2011.142857,12,2,4.628571,162,9,6,6.971429,1.248192,244,1,1,1.0,1,1,0.0,0,1,1,11,11,2,207,203.685714,203,35,9444.542857,4527.744415
1,train_Client_1,37,20629,1207,190,557.540541,197.935960,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,5.135135,10,1,3.128384,2019,2005,2011.837838,8,2,4.324324,160,9,6,7.216216,1.377097,267,1,1,1.0,1,1,0.0,0,1,1,11,11,1,203,203.000000,203,37,14330.756757,6124.126094
2,train_Client_10,18,14375,2400,188,798.611111,513.841374,682,682,0,37.888889,160.748942,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,6.611111,11,2,3.292396,2019,2005,2010.944444,12,4,6.444444,116,9,6,7.055556,1.258955,127,1,1,1.0,1,1,0.0,0,1,1,11,11,2,207,203.222222,203,18,32873.000000,5317.180865
3,train_Client_100,20,24,15,0,1.200000,3.607011,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,5.550000,10,1,3.203206,2012,2005,2008.600000,6,4,4.200000,84,9,6,6.150000,0.670820,123,1,1,1.0,1,1,0.0,0,1,1,11,11,1,413,413.000000,413,4,94.150000,4.120232
4,train_Client_1000,14,9292,800,124,663.714286,224.831365,1468,400,0,104.857143,167.155320,1643,800,0,117.357143,289.433294,514,382,0,36.714286,105.421081,7.571429,12,2,3.390355,2019,2015,2016.857143,4,2,3.714286,52,9,8,8.857143,0.363137,124,1,1,1.0,1,1,0.0,0,1,1,11,11,1,207,207.000000,207,14,6900.428571,4732.111217


In [0]:
train = pd.merge(train_client,agg_trans, on='client_id', how='left')

In [0]:
test_client.head()

,disrict,client_id,client_catg,region,creation_date
0,62,test_Client_0,11,307,28/05/2002
1,69,test_Client_1,11,103,06/08/2009
2,62,test_Client_10,11,310,07/04/2004
3,60,test_Client_100,11,101,08/10/1992
4,62,test_Client_1000,11,301,21/07/1977


In [0]:
test_invoice.head()

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type,year,month
0,test_Client_0,2018-03-16,11,651208,0,203,8,1,755,0,0,0,19145,19900,8,ELEC,2018,3
1,test_Client_0,2014-03-21,11,651208,0,203,8,1,1067,0,0,0,13725,14792,8,ELEC,2014,3
2,test_Client_0,2014-07-17,11,651208,0,203,8,1,0,0,0,0,14792,14792,4,ELEC,2014,7
3,test_Client_0,2015-07-13,11,651208,0,203,9,1,410,0,0,0,16122,16532,4,ELEC,2015,7
4,test_Client_0,2016-07-19,11,651208,0,203,9,1,412,0,0,0,17471,17883,4,ELEC,2016,7


In [0]:
d={"ELEC":0,"GAZ":1}
test_invoice['counter_type']=test_invoice['counter_type'].map(d)

In [0]:
    agg_trans = test_invoice.groupby(['client_id']).agg(aggs)
    agg_trans.columns = ['_'.join(col).strip() for col in agg_trans.columns.values]
    agg_trans.reset_index(inplace=True)

    df = (test_invoice.groupby('client_id')
          .size()
          .reset_index(name='{}transactions_count'.format('1')))

    agg_trans = pd.merge(df, agg_trans, on='client_id', how='left')

In [0]:
test = pd.merge(test_client,agg_trans, on='client_id', how='left')

In [0]:
train.head()

,disrict,client_id,client_catg,region,creation_date,target,1transactions_count,consommation_level_1_sum,consommation_level_1_max,consommation_level_1_min,consommation_level_1_mean,consommation_level_1_std,consommation_level_2_sum,consommation_level_2_max,consommation_level_2_min,consommation_level_2_mean,consommation_level_2_std,consommation_level_3_sum,consommation_level_3_max,consommation_level_3_min,consommation_level_3_mean,consommation_level_3_std,consommation_level_4_sum,consommation_level_4_max,consommation_level_4_min,consommation_level_4_mean,consommation_level_4_std,month_mean,month_max,month_min,month_std,year_max,year_min,year_mean,months_number_max,months_number_min,months_number_mean,months_number_sum,reading_remarque_max,reading_remarque_min,reading_remarque_mean,reading_remarque_std,reading_remarque_sum,counter_coefficient_max,counter_coefficient_min,counter_coefficient_mean,counter_number_nunique,counter_type_nunique,counter_type_mean,counter_type_sum,counter_statue_nunique,tarif_type_nunique,tarif_type_max,tarif_type_min,counter_code_nunique,counter_code_max,counter_code_mean,counter_code_min,old_index_nunique,old_index_mean,old_index_std
0,60,train_Client_0,11,101,31/12/1994,0.0,35,12334,1200,38,352.400000,310.343472,370,186,0,10.571429,43.568935,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,6.685714,12,1,3.587326,2019,2005,2011.142857,12,2,4.628571,162,9,6,6.971429,1.248192,244,1,1,1.0,1,1,0.0,0,1,1,11,11,2,207,203.685714,203,35,9444.542857,4527.744415
1,69,train_Client_1,11,107,29/05/2002,0.0,37,20629,1207,190,557.540541,197.935960,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,5.135135,10,1,3.128384,2019,2005,2011.837838,8,2,4.324324,160,9,6,7.216216,1.377097,267,1,1,1.0,1,1,0.0,0,1,1,11,11,1,203,203.000000,203,37,14330.756757,6124.126094
2,62,train_Client_10,11,301,13/03/1986,0.0,18,14375,2400,188,798.611111,513.841374,682,682,0,37.888889,160.748942,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,6.611111,11,2,3.292396,2019,2005,2010.944444,12,4,6.444444,116,9,6,7.055556,1.258955,127,1,1,1.0,1,1,0.0,0,1,1,11,11,2,207,203.222222,203,18,32873.000000,5317.180865
3,69,train_Client_100,11,105,11/07/1996,0.0,20,24,15,0,1.200000,3.607011,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,5.550000,10,1,3.203206,2012,2005,2008.600000,6,4,4.200000,84,9,6,6.150000,0.670820,123,1,1,1.0,1,1,0.0,0,1,1,11,11,1,413,413.000000,413,4,94.150000,4.120232
4,62,train_Client_1000,11,303,14/10/2014,0.0,14,9292,800,124,663.714286,224.831365,1468,400,0,104.857143,167.155320,1643,800,0,117.357143,289.433294,514,382,0,36.714286,105.421081,7.571429,12,2,3.390355,2019,2015,2016.857143,4,2,3.714286,52,9,8,8.857143,0.363137,124,1,1,1.0,1,1,0.0,0,1,1,11,11,1,207,207.000000,207,14,6900.428571,4732.111217


In [0]:
test.head()

,disrict,client_id,client_catg,region,creation_date,1transactions_count,consommation_level_1_sum,consommation_level_1_max,consommation_level_1_min,consommation_level_1_mean,consommation_level_1_std,consommation_level_2_sum,consommation_level_2_max,consommation_level_2_min,consommation_level_2_mean,consommation_level_2_std,consommation_level_3_sum,consommation_level_3_max,consommation_level_3_min,consommation_level_3_mean,consommation_level_3_std,consommation_level_4_sum,consommation_level_4_max,consommation_level_4_min,consommation_level_4_mean,consommation_level_4_std,month_mean,month_max,month_min,month_std,year_max,year_min,year_mean,months_number_max,months_number_min,months_number_mean,months_number_sum,reading_remarque_max,reading_remarque_min,reading_remarque_mean,reading_remarque_std,reading_remarque_sum,counter_coefficient_max,counter_coefficient_min,counter_coefficient_mean,counter_number_nunique,counter_type_nunique,counter_type_mean,counter_type_sum,counter_statue_nunique,tarif_type_nunique,tarif_type_max,tarif_type_min,counter_code_nunique,counter_code_max,counter_code_mean,counter_code_min,old_index_nunique,old_index_mean,old_index_std
0,62,test_Client_0,11,307,28/05/2002,37,18061,1090,0,488.135135,230.388942,120,120,0,3.243243,19.727878,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,6.837838,12,3,3.337610,2019,2005,2011.648649,8,2,4.378378,162,9,6,6.810811,1.221061,252,1,1,1.0,1,1,0.000000,0,1,1,11,11,1,203,203.000000,203,36,11389.837838,5401.043096
1,69,test_Client_1,11,103,06/08/2009,22,24011,3600,11,1091.409091,739.185437,18549,4053,0,843.136364,986.933891,4011,1144,0,182.318182,316.613838,12899,12899,0,586.318182,2750.076040,6.181818,10,1,2.538023,2019,2009,2014.136364,12,2,4.545455,100,9,6,7.636364,1.216766,168,1,1,1.0,1,1,0.000000,0,2,1,11,11,1,433,433.000000,433,22,29380.772727,17638.422718
2,62,test_Client_10,11,310,07/04/2004,74,40999,1200,0,554.040541,346.775891,2765,400,0,37.364865,95.536205,1165,800,0,15.743243,101.690110,12,12,0,0.162162,1.394972,7.986486,12,4,3.279213,2019,2005,2012.891892,4,4,4.000000,296,9,6,7.459459,1.482216,552,1,1,1.0,2,2,0.432432,32,1,2,40,11,2,207,119.648649,5,74,11774.054054,10987.377541
3,60,test_Client_100,11,101,08/10/1992,40,9774,721,0,244.350000,247.253171,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,5.700000,12,1,3.450752,2012,2005,2008.450000,4,2,3.900000,156,9,6,6.575000,1.034966,263,1,1,1.0,2,2,0.500000,20,1,2,40,11,2,203,104.000000,5,39,13623.725000,13242.361986
4,62,test_Client_1000,11,301,21/07/1977,53,30114,2400,0,568.188679,486.891871,7688,1362,0,145.056604,348.808599,1785,1340,0,33.679245,187.919149,0,0,0,0.000000,0.000000,7.320755,12,2,3.683506,2019,2005,2012.622642,12,2,4.528302,240,9,6,7.905660,1.319443,419,1,1,1.0,3,2,0.339623,18,2,2,40,11,2,203,135.754717,5,52,13927.471698,14371.199312


In [0]:
train.shape,test.shape

((135493, 61), (58069, 60))

In [0]:
for df in [train,test]:
    df['creation_date'] = pd.to_datetime(df['creation_date'])
    df['year'] = df['creation_date'].dt.year
    df['month'] = df['creation_date'].dt.month
    df['month_diff'] = ((datetime.datetime.today() - df['creation_date']).dt.days)//30

In [0]:
col_to_drop = ['client_id', 'creation_date','old_index_std','reading_remarque_std','month_std','consommation_level_1_std', 'consommation_level_2_std', 'consommation_level_3_std','consommation_level_4_std']
for col in col_to_drop:
    if col in train.columns:
        train.drop([col], axis=1, inplace=True)
    if col in test.columns:
        test.drop([col], axis=1, inplace=True)

In [0]:
from sklearn import preprocessing
for f in test.columns:
    if train[f].dtype=='object' or test[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train[f].values) + list(test[f].values))
        train[f] = lbl.transform(list(train[f].values))
        test[f] = lbl.transform(list(test[f].values))  

In [0]:
all_data_na = train.isnull().sum() 
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing ' :all_data_na})
missing_data.head(20)

,Missing


In [0]:
target=train['target']
train.drop('target',axis=1,inplace=True)

In [0]:
import lightgbm
from lightgbm import LGBMRegressor
from lightgbm import LGBMClassifier
model = LGBMClassifier(boosting_type='gbdt', objective='multiclass',
                       num_class=9,num_iteration=1000,num_leaves=31,
                       is_enable_sparse='true',tree_learner='data',min_data_in_leaf=600,max_depth=7,
                       learning_rate=0.04, n_estimators=675, max_bin=255, subsample_for_bin=50000, 
                       min_split_gain=5, min_child_weight=5, min_child_samples=10, subsample=0.995, 
                       subsample_freq=1, colsample_bytree=1, reg_alpha=0, 
                       reg_lambda=0, seed=0, nthread=-1, silent=True)

#Fit to training data
%time model.fit(train,target)

CPU times: user 1min 8s, sys: 58.9 ms, total: 1min 8s
Wall time: 1min 8s


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1,
               importance_type='split', is_enable_sparse='true',
               learning_rate=0.04, max_bin=255, max_depth=7,
               min_child_samples=10, min_child_weight=5, min_data_in_leaf=600,
               min_split_gain=5, n_estimators=675, n_jobs=-1, nthread=-1,
               num_class=9, num_iteration=1000, num_leaves=31,
               objective='multiclass', random_state=None, reg_alpha=0,
               reg_lambda=0, seed=0, silent=True, subsample=0.995,
               subsample_for_bin=50000, subsample_freq=1, tree_learner='data')

In [0]:
import xgboost as xgb
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
clf = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=9,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=2019,
)
%time clf.fit(train, target)

CPU times: user 6min 18s, sys: 188 ms, total: 6min 19s
Wall time: 6min 19s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9, gamma=0,
              learning_rate=0.05, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=2019,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.9, verbosity=1)

In [0]:
pred=clf.predict_proba(test)

In [0]:
pred1=model.predict_proba(test)

In [0]:
preds=pred*0.6+pred1*0.4

ValueError: ignored

In [0]:
preds = pd.DataFrame(preds)

In [0]:
preds.head()

,0,1
0,0.979505,0.020495
1,0.885736,0.114264
2,0.987590,0.012410
3,0.997316,0.002684
4,0.890530,0.109470


In [0]:

submission = pd.DataFrame({
        "client_id": sub["client_id"],
        "target": preds[1]
    })
submission.to_csv('steg1.csv', index=False)



NameError: ignored

In [0]:
submission.head()

,client_id,target
0,test_Client_0,0.020495
1,test_Client_1,0.114264
2,test_Client_10,0.012410
3,test_Client_100,0.002684
4,test_Client_1000,0.109470
